In [8]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # 1 input image channel,
        # 6 output channel,
        # 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation
        self.fc1 = nn.Linear(61 * 61 * 16, 4096)
        self.fc2 = nn.Linear(4096, 256)
        self.fc3 = nn.Linear(256, 2)
        
    def forward(self, x):
        # max pooling over a (2, 2) windows
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
model = Model()
print(model)

Model(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=59536, out_features=4096)
  (fc2): Linear(in_features=4096, out_features=256)
  (fc3): Linear(in_features=256, out_features=2)
)


In [22]:
params = list(model.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [23]:
batch_size = 4
input = Variable(torch.randn(4, 1, 256, 256))
out = model(input)
print(out)

Variable containing:
1.00000e-02 *
 -2.3375  5.4296
 -3.3142  6.7709
 -1.7055  6.9216
 -3.1904  5.9948
[torch.FloatTensor of size 4x2]



In [24]:
model.zero_grad()
out.backward(torch.randn(4, 2))#4 = batch_size #2 = number of classes